# **Project:** Pathology Detection in Crop Plants

Members:
* Domenico Azzarito​
* Guillermo Bajo Laborda​
* Laura Alejandra Moreno​
* Arian Gharehmohammadzadehghashghaei​
* Michele Pezza


*Fundamentals of Data Science | Sapienza University of Rome*

In [ ]:
#imports
import tensorflow as tf
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

## 1. EAD

## 2. CNN

In this step, the image data has been loaded, and also a normaliation, resizing and augmentation process has been implemented.

The key libraries used were TensorFlow for image processing, Pandas for handling the CSV files, and also os library. 



In [19]:
#data
data_dir = 'images/'  
sample_submission_csv = 'sample_submission.csv'
test_csv = 'test.csv'
train_csv = 'train.csv' 
train_df = pd.read_csv(train_csv)
print(train_df.columns)
train_df.columns = train_df.columns.str.strip()
print(train_df.head())

Index(['image_id', 'healthy', 'multiple_diseases', 'rust', 'scab'], dtype='object')
  image_id  healthy  multiple_diseases  rust  scab
0  Train_0        0                  0     0     1
1  Train_1        0                  1     0     0
2  Train_2        1                  0     0     0
3  Train_3        0                  0     1     0
4  Train_4        1                  0     0     0


In [20]:
# Convert the encoded labels into arrays
def con_process_labels(df):
    labels = df[['healthy', 'multiple_diseases', 'rust', 'scab']].values
    return labels
    

In [21]:
#Decode function to load and preprocess the image file

def decode_image(filename, label=None, image_size=(512, 512)):
    filepath = tf.strings.join([data_dir, filename])  
    bits = tf.io.read_file(filepath)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.image.resize(image, image_size)
    print(f"decoded image shape: {image.shape}")
    image = tf.image.convert_image_dtype(image, tf.float32) 
    
    if label is None:
        return image
    else:
        return image, label



In [22]:
#Data augmentation to the images

def data_augmentation(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    
    if label is None:
        return image
    else:
        return image, label

In [23]:
#Create a Tensorflow dataset for testing

def prepare_dataset(df, image_size=(512, 512), batch_size=32, augment=False, is_train=True):
    file_paths = df['image_id'] + '.jpg'  
    
    if is_train:
        labels = con_process_labels(df) 
        dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))
        dataset = dataset.map(lambda x, y: decode_image(x, y, image_size))  
    else:
        dataset = tf.data.Dataset.from_tensor_slices(file_paths)
        dataset = dataset.map(lambda x: decode_image(x, label=None, image_size=image_size))
    
    if augment and is_train:
        dataset = dataset.map(data_augmentation)  
    
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)  
    return dataset

In [24]:
# Create train dataset
train_dataset = prepare_dataset(train_df, image_size=image_size, augment=True, is_train=True)

NameError: name 'image_size' is not defined

In [10]:
# Load test data
test_df = pd.read_csv(test_csv)
test_df.columns = test_df.columns.str.strip()  # Clean the column names for test set
test_dataset = prepare_dataset(test_df, image_size=(512, 512), augment=False, is_train=False)

decoded image shape: (512, 512, 3)


In [25]:
#Verification 

for image_batch, label_batch in train_dataset.take(1):
    print(f"Image batch shape: {image_batch.shape}")
    print(f"Label batch shape: {label_batch.shape}")
    
    # Check the min and max values of the entire image batch
    min_val = tf.reduce_min(image_batch).numpy()
    max_val = tf.reduce_max(image_batch).numpy()
    print("Image batch min and max values:", min_val, max_val)
    
    # Check if any image has invalid values (NaN or Inf)
    if tf.reduce_any(tf.math.is_nan(image_batch)):
        print("Warning: There are NaN values in the image batch.")
    if tf.reduce_any(tf.math.is_inf(image_batch)):
        print("Warning: There are Inf values in the image batch.")

    # Start plotting the first 9 images from the batch
    plt.figure(figsize=(10, 10))
    for i in range(9):  
        plt.subplot(3, 3, i+1)
        img = image_batch[i].numpy() 
        img_min = np.min(img)
        img_max = np.max(img)
        print(f"Image {i} range: min={img_min}, max={img_max}")
        
        # Check for NaN or Inf in individual image
        if np.any(np.isnan(img)):
            print(f"Warning: Image {i} contains NaN values.")
        if np.any(np.isinf(img)):
            print(f"Warning: Image {i} contains Inf values.")
        
        # Clip values to [0, 1] for normalization, then rescale to [0, 255]
        img = np.clip(img, 0, 1) 
        img = (img * 255).astype(np.uint8) 
        
        plt.imshow(img)
        plt.title(f"Label: {np.argmax(label_batch[i].numpy())}")
        plt.axis('off')
    
    plt.show()


NameError: name 'train_dataset' is not defined

# 3. Softmax Processing

In this section, we will focus on preprocessing image data and extracting additional features, such as color histograms, to enhance the model's performance. These features will be used alongside the images to train a convolutional neural network (CNN) model capable of classifying images into four categories: healthy, multiple_diseases, rust, and scab. We will then evaluate the model's performance using key metrics such as accuracy, confusion matrix, and F1-score. Ultimately, this approach will integrate both color histograms and images to achieve a more robust and accurate classification.

In [ ]:
#Extract histograms for R, G, B channels and vectorie them

#This function extracts color histograms for the Red, Green, and Blue channels, 
#normalizes the values, and returns them as a combined vector
def extract_color_histogram(image, bins=256):
   
    # Calculate histograms for each color channel (R, G, B)
    r_hist = tf.histogram_fixed_width(image[..., 0], [0.0, 1.0], nbins=bins)
    g_hist = tf.histogram_fixed_width(image[..., 1], [0.0, 1.0], nbins=bins)
    b_hist = tf.histogram_fixed_width(image[..., 2], [0.0, 1.0], nbins=bins)
    
    # Normalize the histograms by dividing by the total number of pixels
    total_pixels = tf.size(image[..., 0], out_type=tf.float32)  
    r_hist = tf.cast(r_hist, tf.float32) / total_pixels
    g_hist = tf.cast(g_hist, tf.float32) / total_pixels
    b_hist = tf.cast(b_hist, tf.float32) / total_pixels
    
    # Concatenate the R, G, B histograms into a single vector
    color_histogram = tf.concat([r_hist, g_hist, b_hist], axis=-1)

    # Ensure values are within [0, 1]
    color_histogram = tf.clip_by_value(color_histogram, 0.0, 1.0)

    return color_histogram



In [ ]:
# Prepare color histograms for a dataset (no labels)

#This function processes all images in the dataframe by reading and 
#resizing them, then extracting the color histograms
def prepare_color_histograms(df, image_size=(500, 500)):
    def process_image(filename):
        filepath = tf.strings.join([data_dir, filename])  
        bits = tf.io.read_file(filepath)
        image = tf.image.decode_jpeg(bits, channels=3)  # Decode the image
        image = tf.image.resize(image, image_size)  # Resize the image
        color_histogram = extract_color_histogram(image)  # Extract the color histogram
        return color_histogram

    file_paths = df['image_id'] + '.jpg'  # Get the file paths for images
    dataset = tf.data.Dataset.from_tensor_slices(file_paths)
    dataset = dataset.map(process_image)  # Apply the process_image function to all file paths
    histograms = [hist.numpy() for hist in dataset]  # Convert the results to numpy arrays
    return histograms   



In [ ]:
# Prepare the one-hot encoded labels

# This function converts the labels into one-hot encoded vectors
def prepare_one_hot_labels(df):
    labels = df[['healthy', 'multiple_diseases', 'rust', 'scab']].values
    return tf.convert_to_tensor(labels, dtype=tf.float32)  # Convert to tensorflow tensor


In [ ]:
# Prepare the image dataset (for training)

#This function prepares a dataset of images for training, decoding and resizing the images
def prepare_image_dataset(df, image_size=(512, 512)):
    def decode_image(filename, image_size=(512, 512)):
        filepath = tf.strings.join([data_dir, filename])  
        bits = tf.io.read_file(filepath)
        image = tf.image.decode_jpeg(bits, channels=3)
        image = tf.image.resize(image, image_size)  # Resize the image
        image = tf.image.convert_image_dtype(image, tf.float32)  # Normalize image to [0, 1]
        return image

    file_paths = df['image_id'] + '.jpg'  # Get the file paths for images
    dataset = tf.data.Dataset.from_tensor_slices(file_paths)
    dataset = dataset.map(lambda x: decode_image(x, image_size))  # Apply image decoding function
    return dataset

In [39]:
# Combine images and histograms into a single dataset

# This function combines the image dataset and histogram dataset into a single dataset, 
# along with the labels for training.
def combine_image_and_histogram_datasets(image_dataset, histograms_dataset, labels_dataset):
    combined_dataset = tf.data.Dataset.zip((image_dataset, histograms_dataset, labels_dataset))
    combined_dataset = combined_dataset.batch(32).prefetch(tf.data.AUTOTUNE)  # Batch and prefetch for better performance
    return combined_dataset

In [ ]:
# Build the CNN model

# This function builds the CNN model, using the input shape of the images.
# It consists of convolutional layers followed by dense layers for classification
def build_model(input_shape):
    model = tf.keras.Sequential([
        # Convolutional layers for feature extraction
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),  # Flatten the output of the convolutional layers
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(4, activation='softmax')  # Output layer for the 4 possible classes
    ])
    return model

In [42]:
# Training the model

#This function compiles and trains the model on the given dataset
def train_model(train_dataset):
    # Build the model
    model = build_model(input_shape=(512, 512, 3))  # Input shape is (512, 512, 3) for images
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Compile the model
    
    # Train the model on the combined dataset (images + histograms)
    history = model.fit(train_dataset, epochs=10)  # Train for 10 epochs
    return model, history

In [44]:
# Evaluate the model

#This function evaluates the trained model on the test dataset
def evaluate_model(model, test_dataset):
    test_loss, test_accuracy = model.evaluate(test_dataset)
    print(f"Test Accuracy: {test_accuracy}")

    # Additional metrics: Confusion matrix and F1-score
    from sklearn.metrics import confusion_matrix, f1_score

    # Get predictions and true labels
    y_pred = model.predict(test_dataset)
    y_true = [labels.numpy() for _, labels in test_dataset]

    # Convert predictions and labels to single values (from one-hot encoding)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(np.array(y_true), axis=1)

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:")
    print(cm)

    # F1-score
    f1 = f1_score(y_true, y_pred, average='macro')
    print(f"F1-score: {f1}")

In [ ]:
# Prepare train dataset
#train_histograms = prepare_color_histograms(train_df)
#train_labels = prepare_one_hot_labels(train_df)
#train_histograms_dataset = tf.data.Dataset.from_tensor_slices(train_histograms)
#train_labels_dataset = tf.data.Dataset.from_tensor_slices(train_labels)
#train_dataset = tf.data.Dataset.zip((train_histograms_dataset, train_labels_dataset))
#train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

# Prepare test dataset (no labelS, only histograms)
#test_df = pd.read_csv(test_csv)
#test_df.columns = test_df.columns.str.strip()
#test_histograms = prepare_color_histograms(test_df)
#test_histograms_dataset = tf.data.Dataset.from_tensor_slices(test_histograms)
#test_dataset = test_histograms_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

#for histograms, labels in train_dataset.take(1):
#    print("Histograms:", histograms.numpy()[:5])
#    print("Labels:", labels.numpy()[:5])

NotFoundError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to MapDataset:11 transformation with iterator: Iterator::Root::ParallelMapV2: NewRandomAccessFile failed to Create/Open: images/Train_0.jpg : El sistema no puede encontrar el archivo especificado.
; No such file or directory
	 [[{{node ReadFile}}]] [Op:IteratorGetNext] name: 